In [122]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [171]:
import numpy as np
from scipy import stats
from statsmodels.stats.contingency_tables import mcnemar
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix
from scipy.stats import chi2_contingency
from scipy.stats import binom


In [187]:
Baseline_E = np.array([0.231, 0.231, 0.192, 0.308, 0.231, 0.115, 0.240, 0.280, 0.280, 0.160])
Logistic_E = np.array([0.231, 0.192, 0.308, 0.346, 0.154, 0.231, 0.240, 0.280, 0.280, 0.320])
CT_E = np.array([0.269, 0.115, 0.192, 0.231, 0.077, 0.154, 0.200, 0.200, 0.280, 0.120])

In [188]:
def McNemar_t(first_model, stardard_model, n):
    
    n_11 = sum((1 - first_model) + (1 - stardard_model))*25  # *25 is because of the outer fold contains approximately 25 data smaples to let us test 
                                                                #hence we cannot use the standardize ones here if we would like to use the McNerma's package
    n_12 = sum((1 - first_model) + stardard_model)*25
    n_21 = sum(first_model + (1 - stardard_model))*25
    n_22 = sum(first_model + stardard_model)*25
    
    confusion_matrix = [
        [n_11, n_12],
        [n_21, n_22]
    ]
    
    
    # McNemar's test
    result = mcnemar(confusion_matrix, exact=True)
    
    E_theta = (n_12 - n_21)/(n*25)
    
    # Calculate the confidence interval based on chi-squared distribution
    alpha = 0.05  # significance level 95% CI
    
    # Find the critical values for the confidence interval
    #theata_L_value = chi2_contingency(confusion_matrix, alpha / 2)[0]
    #theata_U_value = chi2_contingency(confusion_matrix, 1 - alpha / 2)[0]

    
    # Calculate the confidence interval
    #theata_L = 0.5 * (1 - np.sqrt(1 - theata_L_value / (n_11 + n_12)))
    #theata_U = 0.5 * (1 + np.sqrt(1 - theata_U_value / (n_11 + n_12)))
    
    alpha = 0.05  # 95% CI
    Q = (n**2 * (n+1)*(E_theta + 1)*(1 - E_theta)) / (n*(n_12 + n_21) - (n_12 - n_21)**2)
    f = (E_theta+1)/2 * (Q - 1)
    g = (1 - E_theta)/2 * (Q - 1)
    
    theta_L = 2 * binom.ppf(alpha/2, f, g) - 1
    theta_U = 2 * binom.ppf(1 - alpha/2, f, g) - 1
    

    print("McNemar's test statistic:", result.statistic)
    print("p-value:", result.pvalue)
    print("E_theata:", E_theta)
    print(f"Confidence interval: ({theta_L}, {theta_U})")

<h1>Compare Baseline model and Logistic Regression model<h1>

In [189]:
print(Baseline_E)
print(Logistic_E)

[0.231 0.231 0.192 0.308 0.231 0.115 0.24  0.28  0.28  0.16 ]
[0.231 0.192 0.308 0.346 0.154 0.231 0.24  0.28  0.28  0.32 ]


In [190]:
n = 10
McNemar_t(Logistic_E, Baseline_E, n)

McNemar's test statistic: 242.15
p-value: 0.5023743407533191
E_theata: -0.06280000000000006
Confidence interval: (nan, nan)


<h1>Compare Logistic Regression  and Classification Trees Values<h1>

In [191]:
print(CT_E)
print(Logistic_E)

[0.269 0.115 0.192 0.231 0.077 0.154 0.2   0.2   0.28  0.12 ]
[0.231 0.192 0.308 0.346 0.154 0.231 0.24  0.28  0.28  0.32 ]


In [192]:
# We use the Classification Trees  as a standard for Logistic Regression
n = 10
McNemar_t(CT_E, Logistic_E, n)

McNemar's test statistic: 231.4
p-value: 0.09789002802762435
E_theata: 0.14880000000000007
Confidence interval: (nan, nan)


<h1>Compare Classification Trees Values and Baseline model<h1>

In [193]:
print(CT_E)
print(Baseline_E)

[0.269 0.115 0.192 0.231 0.077 0.154 0.2   0.2   0.28  0.12 ]
[0.231 0.231 0.192 0.308 0.231 0.115 0.24  0.28  0.28  0.16 ]


In [194]:
n = 10
McNemar_t(CT_E, Baseline_E, n)

McNemar's test statistic: 239.25
p-value: 0.3476641793130139
E_theata: 0.086
Confidence interval: (nan, nan)
